In [ ]:

print ('this area is more for dataframe reviews ')
import pandas as pd
import pyodbc


In [ ]:
import os
from sqlalchemy import create_engine
import urllib.parse
%env DB_HOST=ShopMgt
%env DB_USER=SA
%env DB_SERVER=localhost
%env DB_PORT=1433
%env DB_PASSWORD=8189180941sS@
server = os.environ.get('DB_SERVER')  
database = os.environ.get('DB_HOST')  
username = os.environ.get('DB_USER') 
password = os.environ.get('DB_PASSWORD')

connect_string='DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password+';TrustServerCertificate=yes;Encrypt=yes;'
sql_alchemy_string=connect_string

In [ ]:
from sqlalchemy import create_engine

In [ ]:
from sqlalchemy import insert

In [ ]:
# add this one on the cell to test if there is no output
{
    "tags": [
        "hide-output"
    ]
}


In [ ]:
df=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/PhoneNum_Mitchell1_20221222.json')

In [ ]:
%env

In [ ]:
df.info()


In [ ]:
df.head()


In [ ]:
df.index


In [ ]:
#df['AppointmentRequestUid']

In [ ]:
df.duplicated()


In [ ]:
df.iloc[0,1]

In [ ]:
#df.iloc[1,3].d

In [ ]:
#from dotenv import load_dotenv


In [ ]:
sql_alchemy_string


In [ ]:
## failed  thee first time
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
# need to add ;TrustServerCertificate=yes;Encrypt=no;

In [ ]:
df.count()


In [ ]:
# connect_string='DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';TrustServerCertificate=yes;Encrypt=yes;'


In [ ]:
connect_string

In [ ]:
conn=pyodbc.connect(connect_string)

In [ ]:
cursor = conn.cursor()

In [ ]:
cursor.execute("SELECT @@version;") 

In [ ]:
row=cursor.fetchone()


In [ ]:
row


In [ ]:
row[0]

In [ ]:
df.info()

In [ ]:
for index, row in df.head().iterrows():
    print(index, row)

In [ ]:
dfhead=df.head()


In [ ]:
dfhead.dtypes

In [ ]:
dfhead.Note

In [ ]:
df=df.convert_dtypes()

In [ ]:
df.dtypes

In [ ]:
df['RepairOrderId'].value_counts(dropna=False)

In [ ]:
df.LastChangeDate

In [ ]:
%timeit pd.to_datetime(dfhead.LastChangeDate,infer_datetime_format=True)

In [ ]:
pd.to_datetime(dfhead.LastChangeDate,infer_datetime_format=True, errors='ignore')

In [ ]:
dfhead.LastChangeDate

In [ ]:
start=pd.to_datetime('2022-12-22 22:56:03.877')

In [ ]:
import urllib.parse
# password_2=urllib.parse.quote_plus(password)

In [ ]:
#---------------------------2023-01-01 creating tables ----------------------------

In [ ]:
print('preparing to insert data into customers_new_01 table')

In [ ]:
json_file_path = '/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/Customer_Mitchell1_20221222.json'

In [ ]:
df_cust=pd.read_json(json_file_path)
df_cust=df_cust.convert_dtypes()
df_cust.info()

In [ ]:
print ('3-step: 1-slice; 2-rename column names;3 to_sql into the database table')

In [ ]:
df_cust_cut=df_cust.loc[:,["CustId","TitleId", "FirstName", "LastName", "DOB", "SpouseName",
       "ChargeOK", "Remarks",
       "ResaleNum", "IncludeInCRMCampaign", 
       "TaxExempt", "LastVisited", "FirstVisited",
       "RefTypeId",
       "IsDeleted",  "IsFromSchedule", "MembershipNo",
       "SMSAllowed", "EmailAddress",
       "LastChangeDate", "FleetVendorId"]]

In [ ]:
df_cust_cut

In [ ]:
df_cust_cut.info()

In [ ]:
df_cust_cut.columns

In [ ]:
df_cust_final=df_cust_cut.rename(columns={'CustId':'customer_id', 'TitleId':'customer_title_id',
        'FirstName':'customer_first_name',
        'LastName':'customer_last_name', 'DOB':'customer_dob', 'SpouseName':'customer_spouse_name',
       'ChargeOK':'customer_is_okay_to_charge', 'Remarks':'customers_memo_1', 'ResaleNum':'customer_resale_permit_nbr',
       'IncludeInCRMCampaign':'customer_is_in_social_crm', 'TaxExempt':'customer_is_tax_exempt', 
        'LastVisited':'customer_last_visit_date', 'FirstVisited':'customer_first_visit_date',
       'RefTypeId':'customer_hear_from_us_type', 'IsDeleted':'customer_is_activate', 
        'IsFromSchedule':'customer_is_created_from_appointments', 'MembershipNo':'customer_memebership_nbr',
       'SMSAllowed':'customer_does_allow_SMS', 'EmailAddress':'customer_email_address_in_json',
       'LastChangeDate':'customer_last_updated_datetime', 'FleetVendorId':'customer_fleet_vendor_id'})

In [ ]:
# the final table shold have 21 columns. There are 5 new new columns that have to be definfied.
df_cust_final.info()

In [ ]:
df_cust_final

In [ ]:
df_cust_final.customer_title_id.value_counts()

In [ ]:
print ('reverse the is_deleted values to is_activate. reverse the boolean value! IMPORTANT')
for index, row in df_cust_final.iterrows():
    if row.customer_is_activate == 1:
        row.customer_is_activate == 0
    else:
        row.customer_is_activate == 1

In [ ]:
print(sql_alchemy_string)

In [ ]:
db_engine_cust=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
args, kwargs = db_engine_cust.dialect.create_connect_args(db_engine_cust.url)
#list the argument and key arguments of the 
args,kwargs

In [ ]:
with db_engine_cust.connect() as conn:
    conn.execute(("USE prolube76DBSTG01"))
    df_cust_final.to_sql('customers_new_01',conn, index=False,if_exists='append')

In [ ]:
# db_enging_v2=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string_v2}', echo=True)#fast_executemany=True

In [ ]:
# this one returns an error still.
with db_enging_v2.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    result = conn.execute(
            insert('dbo.customers_new_01'),
            df_cust_final,
    )
    conn.commit()

In [ ]:
with db_enging_v2.connect() as conn:
    for index, row in df_cust_final.iterrows():
        conn.execute(f"INSERT INTO customers_new_01(customer_id,row.customer_title_id,customer_first_name,customer_last_name,customer_dob)"
#                     "customer_spouse_name,customer_is_okay_to_charge,customers_memo_1,customer_resale_permit_nbr,customer_is_in_social_crm"
#                     "customer_is_tax_exempt,customer_last_visit_date,first_visit_date,customer_is_created_from_appointments,customer_memebership_nbr"
#                     "customer_does_allow_SMS,customer_email_address_in_json,customer_fleet_vendor_id,customer_last_updated_datetime) "
                     "values({row.customer_id},{row.customer_title_id},{row.customer_first_name},{row.customer_last_name},{row.customer_dob}"
#                     ",{row.customer_spouse_name},{row.customer_is_okay_to_charge},"
#                     "{row.customers_memo_1},{row.customer_resale_permit_nbr},{row.customer_is_in_social_crm},"
#                     "{row.customer_is_tax_exempt},{row.customer_last_visit_date},{row.customer_first_visit_date},"
#                     "{row.customer_is_created_from_appointments},{row.customer_memebership_nbr},{row.customer_does_allow_SMS},"
#                     "{row.customer_email_address_in_json},{row.customer_fleet_vendor_id},{row.customer_last_updated_datetime},)"
                    )
    conn.commit()

In [ ]:
df_cust_final.columns

In [ ]:

# try the pyodbc method now
cursor

In [ ]:
for index, row in df_cust_final.iterrows():
     cursor.execute("INSERT INTO customers_new_01 (customer_id,customer_title_id,customer_first_name,customer_last_name) values(?,?,?,?)",
                    row.customer_id,
                    row.customer_title_id,
                    row.customer_first_name,
                    row.customer_last_name)
cnxn.commit()


In [ ]:
df_cust_final2=df_cust_final.convert_dtypes()
#this is the most used method as it automatically converts the column types to best possible types.

In [ ]:
df_cust_final2.info()

In [ ]:
for index, row in df_cust_final2.iterrows():
     cursor.execute("INSERT INTO customers_new_01 (customer_id,customer_title_id,customer_first_name,customer_last_name) values(?,?,?,?)",
                    row.customer_id,
                    row.customer_title_id,
                    row.customer_first_name,
                    row.customer_last_name)
cnxn.commit()


In [ ]:

## starting new table phone_new_01
print ('starting to import data into a new table phone_new_01')

In [ ]:
df_phone=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/PhoneNum_Mitchell1_20221222.json')

In [ ]:
df_phone.info()


In [ ]:
df_phone=df_phone.convert_dtypes()
df_phone.info()

In [ ]:
# step 1: slice the dataframe and retain only the wanted columns
df_phone_cut=df_phone.loc[:,['PhoneId','PhoneDescId','PhoneNum','Ext','Display','Note','LastChangeDate']]

In [ ]:
df_phone_cut.info()

In [ ]:
df_phone_final=df_phone_cut.rename(columns={'PhoneId':'phone_id',
                                          'PhoneDescId':'phone_id',
                                          'PhoneNum':'phone_number',
                                          'Ext':'phone_number_ext',
                                          'Display':'phone_displayed_name',
                                          'Note':'phone_memo_01',
                                          'LastChangeDate':'phone_last_updated_date'})

In [ ]:
df_phone_final.info()




In [ ]:
# sql_alchemy_string=sql_alchemy_string_v2

In [ ]:
sql_alchemy_string

In [ ]:
db_engine_phone=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
# use the prolube76DBSTG01; 
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table
with db_engine_phone.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_phone_final.to_sql('phones_new_01',conn, index=False,if_exists='append')

In [ ]:
#-----vehicles_new_01---------

In [ ]:
print ('preparing to insert vehicles_new_01')

In [ ]:
df_vehicle=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/Vehicle_Mitchell1_20221222.json')

In [ ]:
df_vehicle.info()

In [ ]:
df_vehicle=df_vehicle.convert_dtypes()
df_vehicle.info()

In [ ]:
df_vehicle_cut=df_vehicle.loc[:,[ 'VehicleId','CustId','Year','MakeId','SubModelId','BodyId','EngineId','TransmissionId'
,'BrakeId','DriveTypeId','GVWId','Odometer1','Odometer2','Vin','InspDate','LastInDate','License','LicenseState',
'PartLevel','LaborLevel','UseVehicleLevels','VehicleMemo','VehicleMemoPrintOnOrder','IncludeInCRMCampaign'
,'Color','Deleted','VehicleClass','DriverPhoneId','EngineHoursIn','EngineHoursOut'
,'ActiveRecallCount','ActiveRecallLastChecked','LastChangeDate']]

In [ ]:
df_vehicle_cut.info()

In [ ]:
# now rename the column names. Vehicle_Final

df_vehicle_final = df_vehicle_cut.rename(
    columns={'VehicleId':'vehicle_id','CustId':'vehicle_cust_id','Year':'vehicle_year','MakeId':'vehicle_make_id'
    ,'SubModelId':'vehicle_sub_model_id','BodyId':'vehicle_body_style_id','EngineId':'vehicle_engine_id','TransmissionId':'vehicle_transmission_id'
    ,'BrakeId':'vehicle_brake_id','DriveTypeId':'vehicle_drive_type_id','GVWId':'vehicle_GVW_id','Odometer1':'vehicle_odometer_1','Odometer2':'vehicle_odometer_2','Vin':'VIN_number'
    ,'InspDate':'vehicle_inspection_datetime','LastInDate':'vehicle_last_in_date','License':'vehicle_license_plate_nbr','LicenseState':'vehicle_license_state'
    ,'PartLevel':'vehicle_part_level','LaborLevel':'vehicle_labor_level','UseVehicleLevels':'vehicle_used_level','VehicleMemo':'vehicle_memo_01','VehicleMemoPrintOnOrder':'vehicle_memo_does_print_on_order','IncludeInCRMCampaign':'vehicle_is_included_in_CRM_compaign'
    ,'Color':'vehicle_color','Deleted':'vehicle_record_is_activate','VehicleClass':'vehicle_class_id','DriverPhoneId':'vehicle_phone_id','EngineHoursIn':'vehicle_engine_hour_in','EngineHoursOut':'vehicle_engine_hour_out'
    ,'ActiveRecallCount':'vehicle_active_recall_counts','ActiveRecallLastChecked':'vehicle_recall_last_checked_datetime','LastChangeDate':'vehicle_last_updated_datetime'})

In [ ]:

df_vehicle_final.info()



In [ ]:
print ('reverse the is_deleted values to is_activate. reverse the boolean value')
for index, row in df_vehicle_final.iterrows():
    if row.vehicle_record_is_activate==1:
        row.vehicle_record_is_activate == 0
    else:
        row.vehicle_record_is_activate == 1

In [ ]:
# the result should have 33 columns. the total rows are 44578 entries
df_vehicle_final.info()

In [ ]:
sql_alchemy_string=connect_string

In [ ]:
db_engine_vehicle=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
# use the prolube76DBSTG01; 
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table
with db_engine_vehicle.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_vehicle_final.to_sql('vehicles_new_01',conn, index=False,if_exists='append')

In [ ]:
# list the argument and key arguments
args, kwargs = db_engine_vehicle.dialect.create_connect_args(db_engine_vehicle.url)


In [ ]:
args, kwargs

In [ ]:
aa=df_vehicle_final[df_vehicle_final.vehicle_id==22018]

In [ ]:
aa.vehicle_license_state

In [ ]:
df_vehicle_final.info()

In [ ]:
# check out the final vechile dataframe that feeds into the sql data table
df_vehicle_final.info()

In [ ]:
print('start to import repair orders repairorders_new_01')


In [ ]:
json_file_path = '/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/RepairOrder_Mitchell1_20221222.json'
print(f'the repair order json file is {json_file_path}')
df_repairorder=pd.read_json(json_file_path)
print(df_repairorder.info())

In [ ]:
print('convert the column dtypes into the best possible ones using convert_dtypes()')
df_repairorder=df_repairorder.convert_dtypes()

In [ ]:
df_repairorder.info()

In [ ]:
df_repairorder.columns

In [ ]:
# slice the dataframe. first step
df_repairorder_cut=df_repairorder.loc[:, ["RepairOrderId" ,
"RepairOrderPhaseId" ,
"CustId" ,
"VehicleId" ,
"Location" ,
"StatusDescription" ,
"ScheduleDate" ,
"ScheduledHours" ,
"PromiseDate" ,
"RoPrinted" ,
"InvoicePrinted" ,
"TimeIn" ,
"TimeOut" ,
"Hat" ,
"DatePosted" ,
"OdometerIn" ,
"OdometerOut" ,
"ReferenceNumber" ,
"PrintedDate" ,
"TaxExempt" ,
"Notes" ,
"Observations" ,
"CreatedAsEstimate" ,
"MarginPct" ,
"HazWasteAmt" ,
"LaborSale" ,
"PartsSale" ,
"ShopSuppliesAmt" ,
"TaxAmtHazMat" ,
"TaxAmtShopSupplies" ,
"TotalTaxAmt" ,
"BalanceDueAdjustment" ,
"DiscountAmt" ,
"PartDiscountDescriptionId" ,
"LaborRateDescriptionId" ,
"OrderTotal" ,
"CalculateHazWasteCost" ,
"CalculateShopSuppliesCost" ,
"EngineHoursIn" ,
"EngineHoursOut" ,
"LastChangeDate" ,
"AppointmentRequestUid" ,]]

In [ ]:
print(df_repairorder_cut.info())

In [ ]:
# now rename the column names. repairorder final

df_repairorder_final= df_repairorder_cut.rename(
    columns={"RepairOrderId":"repair_order_id" ,
"RepairOrderPhaseId":"repair_order_phase_id" ,
"CustId": "repair_order_cust_id" ,
"VehicleId": "repair_order_vehicle_id",
"Location" :"repair_order_serviced_vehicle_location" ,
"StatusDescription":"repair_order_service_status" ,
"ScheduleDate":"repair_order_scheduled_start_datetime" ,
"ScheduledHours":"repair_order_billed_hours" ,
"PromiseDate":"repair_order_promise_datetime" ,
"RoPrinted":"repair_order_is_printed" ,
"InvoicePrinted":"repair_order_invoice_is_printed" ,
"TimeIn":"repair_order_serviced_vehicle_in_datetime"  ,
"TimeOut":"repair_order_serviced_vehicle_out_datetime" ,
"Hat":"repair_order_serviced_vehicle_hat"  ,
"DatePosted": "repair_order_posted_datetime" ,
"OdometerIn" :"repair_order_serviced_vehicle_odometer_in",
"OdometerOut": "repair_order_odometer_out" ,
"ReferenceNumber": "repair_order_reference_number" ,
"PrintedDate" : "repair_order_receipt_printed_datetime" ,
"TaxExempt": "repair_order_snapshot_is_tax_exempt" ,
"Notes": "repair_order_aggr_notes" ,
"Observations": "repair_order_observation_text_area" ,
"CreatedAsEstimate": "repair_order_created_as_estimate"  ,
"MarginPct": "repair_order_snapshot_margin_pct" ,
"HazWasteAmt": "repair_order_snapshot_haz_waste_amount",
"LaborSale":"repair_order_snapshot_labor_sale_amount" ,
"PartsSale":"repair_order_snapshot_parts_sale_amount" ,
"ShopSuppliesAmt":"repair_order_snapshot_supply_from_shop_amount" ,
"TaxAmtHazMat":"repair_order_snapshot_tax_haz_material_amount" ,
"TaxAmtShopSupplies":"repair_order_snapshot_tax_supply_from_shop_amount" ,
"TotalTaxAmt":"repair_order_snapshot_total_tax_amount" ,
"BalanceDueAdjustment": "repair_order_snapshot_balance_due_adjusted" ,
"DiscountAmt": "repair_order_snapshot_discounted_amount",
"PartDiscountDescriptionId": "repair_order_snapshot_part_discounted_desc_id",
"LaborRateDescriptionId":"repair_order_snapshot_labor_discounted_desc_id" ,
"OrderTotal":"repair_order_snapshot_order_total_amount" ,
"CalculateHazWasteCost":"repair_order_snapshot_calc_haz_waste_cost"  ,
"CalculateShopSuppliesCost":"repair_order_snapshot_calc_shop_supply_cost" ,
"EngineHoursIn":"repair_order_serviced_vehcle_engine_hours_in",
"EngineHoursOut":"repair_order_serviced_vehcle_engine_hours_out" ,
"LastChangeDate":"repair_order_last_updated_datetime",
"AppointmentRequestUid":"repair_order_appointment_request_uid" })

In [ ]:
 df_repairorder_final.info()

In [ ]:
db_engine_repairorder=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
db_engine_repairorder

In [ ]:
# list the argument and key arguments
args, kwargs = db_engine_vehicle.dialect.create_connect_args(db_engine_vehicle.url)
args, kwargs

In [ ]:
# use the prolube76DBSTG01; insert data into repairoders_new_01 table
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table
with db_engine_repairorder.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_repairorder_final.to_sql('repairorders_new_01',conn, index=False,if_exists='append')

In [ ]:
# preparing the estimates_new_01 table
import pandas as pd


In [ ]:
json_file_path = '/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/estimate_Mitchell1_20221222.json'
df_estimate = pd.read_json(json_file_path)
df_estimate = df_estimate.convert_dtypes()

In [ ]:
df_estimate.info()

In [ ]:
df_estimate.columns

In [ ]:
df_estimate_cut=df_estimate.loc[:, ['EstimateId', 'RepairOrderId', 
       'EstimatedLaborCost','EstimatedPartsCost', 'EstimatedHours', 'EstimatedTaxAmount',
       'EstimatedHazMatCost', 'EstShopSuppliesCost', 'EstimatedDiscount',
       'LastChangeDate']]

In [ ]:
df_estimate_final=df_estimate_cut.rename(
    columns={'EstimateId':'estimate_id', 
             'RepairOrderId':'estimate_repair_order_id', 
             'EstimatedLaborCost':'estimate_snapshot_labor_cost',
             'EstimatedPartsCost':'estimate_snapshot_parts_cost', 
             'EstimatedHours':'estimate_billed_hours', 'EstimatedTaxAmount':'estimate_snapshot_tax_amount',
             'EstimatedHazMatCost':'estimate_snapshot_haz_mat_amount',
             'EstShopSuppliesCost':'estimate_snapshot_shop_supply_cost', 
             'EstimatedDiscount':'estimate_snapshot_discount_amount',
             'LastChangeDate':'estimate_last_updated_date'
        
    }
)









In [ ]:
df_estimate_final=df_estimate_final.convert_dtypes()
df_estimate_final.info()

In [ ]:
import os
from sqlalchemy import create_engine
import urllib.parse
%env DB_HOST=ShopMgt
%env DB_USER=SA
%env DB_SERVER=localhost
%env DB_PORT=1433
%env DB_PASSWORD=8189180941sS@
server = os.environ.get('DB_SERVER')  
database = os.environ.get('DB_HOST')  
username = os.environ.get('DB_USER') 
password = os.environ.get('DB_PASSWORD')

connect_string='DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password+';TrustServerCertificate=yes;Encrypt=yes;'
sql_alchemy_string=connect_string

In [ ]:
sql_alchemy_string

In [ ]:
db_engine_estimate=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
# create new estimate sql table

In [ ]:
# insert data into estimates_new_01 table
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table
with db_engine_estimate.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_estimate_final.to_sql('estimates_new_01',conn, index=False,if_exists='append')

In [ ]:
file_name_list = ['Customer', 'PhoneNum', 'vehicle','RepairOrder','estimate','Schedule','email']

In [ ]:
for index,file in  enumerate(file_name_list):
    print (f'the index is {index}; the value of the file item is {file}')

In [8]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import insert, create_engine

load_dotenv()

sql_alchemy_string = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER=' + os.environ.get('DB_SERVER')+';DATABASE='+os.environ.get('DB_HOST')+';UID='+os.environ.get('DB_USER')+';PWD='+os.environ.get('DB_PASSWORD')+';TrustServerCertificate=yes;Encrypt=yes;'
# 

In [ ]:
# create a sql alchemy engine (based on pyodbc).
json_file_path = '/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/Customer_Mitchell1_20221222.json'
df_cust=pd.read_json(json_file_path)
df_cust=df_cust.convert_dtypes()
#df_cust.info()
df_cust_cut=df_cust.loc[:,["CustId","TitleId", "FirstName", "LastName", "DOB", "SpouseName",
       "ChargeOK", "Remarks",
       "ResaleNum", "IncludeInCRMCampaign", 
       "TaxExempt", "LastVisited", "FirstVisited",
       "RefTypeId",
       "IsDeleted",  "IsFromSchedule", "MembershipNo",
       "SMSAllowed", "EmailAddress",
       "LastChangeDate", "FleetVendorId"]]

df_cust_final=df_cust_cut.rename(columns={'CustId':'customer_id', 'TitleId':'customer_title_id',
        'FirstName':'customer_first_name',
        'LastName':'customer_last_name', 'DOB':'customer_dob', 'SpouseName':'customer_spouse_name',
       'ChargeOK':'customer_is_okay_to_charge', 'Remarks':'customers_memo_1', 'ResaleNum':'customer_resale_permit_nbr',
       'IncludeInCRMCampaign':'customer_is_in_social_crm', 'TaxExempt':'customer_is_tax_exempt', 
        'LastVisited':'customer_last_visit_date', 'FirstVisited':'customer_first_visit_date',
       'RefTypeId':'customer_hear_from_us_type', 'IsDeleted':'customer_is_activate', 
        'IsFromSchedule':'customer_is_created_from_appointments', 'MembershipNo':'customer_memebership_nbr',
       'SMSAllowed':'customer_does_allow_SMS', 'EmailAddress':'customer_email_address_in_json',
       'LastChangeDate':'customer_last_updated_datetime', 'FleetVendorId':'customer_fleet_vendor_id'})

for index, row in df_cust_final.iterrows():
    if row.customer_is_activate == 1:
        row.customer_is_activate == 0
    else:
        row.customer_is_activate == 1

db_engine_cust=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
with db_engine_cust.connect() as conn:
    conn.execute(("USE prolube76DBSTG01"))
    df_cust_final.to_sql('customers_new_01',conn, index=False,if_exists='append')


In [ ]:
# insert the customers table. Original data.
with db_engine_cust.connect() as conn:
    conn.execute(("USE prolube76DBSTG01"))
    df_cust.to_sql('customers',conn, index=False,if_exists='append')

In [ ]:
df_phone=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/PhoneNum_Mitchell1_20221222.json')
df_phone=df_phone.convert_dtypes()
df_phone_cut=df_phone.loc[:,['PhoneId','PhoneDescId','PhoneNum','Ext','Display','Note','LastChangeDate']]
df_phone_final=df_phone_cut.rename(columns={'PhoneId':'phone_id',
                                          'PhoneDescId':'phone_id',
                                          'PhoneNum':'phone_number',
                                          'Ext':'phone_number_ext',
                                          'Display':'phone_displayed_name',
                                          'Note':'phone_memo_01',
                                          'LastChangeDate':'phone_last_updated_date'})

db_engine_phone=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table


In [ ]:
with db_engine_phone.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_phone_final.to_sql('phones_new_01',conn, index=False,if_exists='append')

In [ ]:
#-------------insert data into vehilces_new_01 table ---

In [4]:
df_vehicle=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/Vehicle_Mitchell1_20221222.json')
df_vehicle = df_vehicle.convert_dtypes()

df_vehicle_cut = df_vehicle.loc[:,[ 'VehicleId','CustId','Year','MakeId','SubModelId','BodyId','EngineId','TransmissionId'
,'BrakeId','DriveTypeId','GVWId','Odometer1','Odometer2','Vin','InspDate','LastInDate','License','LicenseState',
'PartLevel','LaborLevel','UseVehicleLevels','VehicleMemo','VehicleMemoPrintOnOrder','IncludeInCRMCampaign'
,'Color','Deleted','VehicleClass','DriverPhoneId','EngineHoursIn','EngineHoursOut'
,'ActiveRecallCount','ActiveRecallLastChecked','LastChangeDate']]

df_vehicle_final = df_vehicle_cut.rename(
    columns={'VehicleId':'vehicle_id','CustId':'vehicle_cust_id','Year':'vehicle_year','MakeId':'vehicle_make_id'
    ,'SubModelId':'vehicle_sub_model_id','BodyId':'vehicle_body_style_id','EngineId':'vehicle_engine_id','TransmissionId':'vehicle_transmission_id'
    ,'BrakeId':'vehicle_brake_id','DriveTypeId':'vehicle_drive_type_id','GVWId':'vehicle_GVW_id','Odometer1':'vehicle_odometer_1','Odometer2':'vehicle_odometer_2','Vin':'VIN_number'
    ,'InspDate':'vehicle_inspection_datetime','LastInDate':'vehicle_last_in_date','License':'vehicle_license_plate_nbr','LicenseState':'vehicle_license_state'
    ,'PartLevel':'vehicle_part_level','LaborLevel':'vehicle_labor_level','UseVehicleLevels':'vehicle_used_level','VehicleMemo':'vehicle_memo_01','VehicleMemoPrintOnOrder':'vehicle_memo_does_print_on_order','IncludeInCRMCampaign':'vehicle_is_included_in_crm_campaign'
    ,'Color':'vehicle_color','Deleted':'vehicle_record_is_activate','VehicleClass':'vehicle_class_id','DriverPhoneId':'vehicle_phone_id','EngineHoursIn':'vehicle_engine_hour_in','EngineHoursOut':'vehicle_engine_hour_out'
    ,'ActiveRecallCount':'vehicle_active_recall_counts','ActiveRecallLastChecked':'vehicle_recall_last_checked_datetime','LastChangeDate':'vehicle_last_updated_datetime'})


In [5]:
df_vehicle_final=df_vehicle_final.convert_dtypes()
df_vehicle_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44578 entries, 0 to 44577
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   vehicle_id                            44578 non-null  Int64 
 1   vehicle_cust_id                       39478 non-null  Int64 
 2   vehicle_year                          42892 non-null  Int64 
 3   vehicle_make_id                       42746 non-null  Int64 
 4   vehicle_sub_model_id                  42556 non-null  Int64 
 5   vehicle_body_style_id                 42346 non-null  Int64 
 6   vehicle_engine_id                     41966 non-null  Int64 
 7   vehicle_transmission_id               9667 non-null   Int64 
 8   vehicle_brake_id                      23700 non-null  Int64 
 9   vehicle_drive_type_id                 19356 non-null  Int64 
 10  vehicle_GVW_id                        24411 non-null  Int64 
 11  vehicle_odometer_1          

In [6]:
db_engine_vehicle=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [7]:
# then use the pandas builtin to_sql method. index= False means the index of the dataframe won't be inserted into phones_new_01 table
with db_engine_vehicle.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_vehicle_final.to_sql('vehicles_new_01',conn, index=False,if_exists='append')

In [ ]:
df_vehicle_final[df_vehicle_final.vehicle_id==22018]

In [ ]:
with db_engine_vehicle.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_vehicle.to_sql('vehicles',conn, index=False,if_exists='append')

In [2]:
df_repairorder=pd.read_json('/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/RepairOrder_Mitchell1_20221222.json'
)
df_repairorder=df_repairorder.convert_dtypes()
# slice the dataframe. first step
df_repairorder_cut=df_repairorder.loc[:, ["RepairOrderId" ,
    "RepairOrderPhaseId" ,
    "CustId" ,
    "VehicleId" ,
    "Location" ,
    "StatusDescription" ,
    "ScheduleDate" ,
    "ScheduledHours" ,
    "PromiseDate" ,
    "RoPrinted" ,
    "InvoicePrinted" ,
    "TimeIn" ,
    "TimeOut" ,
    "Hat" ,
    "DatePosted" ,
    "OdometerIn" ,
    "OdometerOut" ,
    "ReferenceNumber" ,
    "PrintedDate" ,
    "TaxExempt" ,
    "Notes" ,
    "Observations" ,
    "CreatedAsEstimate" ,
    "MarginPct" ,
    "HazWasteAmt" ,
    "LaborSale" ,
    "PartsSale" ,
    "ShopSuppliesAmt" ,
    "TaxAmtHazMat" ,
    "TaxAmtShopSupplies" ,
    "TotalTaxAmt" ,
    "BalanceDueAdjustment" ,
    "DiscountAmt" ,
    "PartDiscountDescriptionId" ,
    "LaborRateDescriptionId" ,
    "OrderTotal" ,
    "CalculateHazWasteCost" ,
    "CalculateShopSuppliesCost" ,
    "EngineHoursIn" ,
    "EngineHoursOut" ,
    "LastChangeDate" ,
    "AppointmentRequestUid" ,]]
#renaming columns. check the 2022-12-24-new-site-DB-config.xlsx
df_repairorder_final= df_repairorder_cut.rename(
    columns={"RepairOrderId":"repair_order_id" ,
    "RepairOrderPhaseId":"repair_order_phase_id" ,
    "CustId": "repair_order_cust_id" ,
    "VehicleId": "repair_order_vehicle_id",
    "Location" :"repair_order_serviced_vehicle_location" ,
    "StatusDescription":"repair_order_service_status" ,
    "ScheduleDate":"repair_order_scheduled_start_datetime" ,
    "ScheduledHours":"repair_order_billed_hours" ,
    "PromiseDate":"repair_order_promise_datetime" ,
    "RoPrinted":"repair_order_is_printed" ,
    "InvoicePrinted":"repair_order_invoice_is_printed" ,
    "TimeIn":"repair_order_serviced_vehicle_in_datetime"  ,
    "TimeOut":"repair_order_serviced_vehicle_out_datetime" ,
    "Hat":"repair_order_serviced_vehicle_hat"  ,
    "DatePosted": "repair_order_posted_datetime" ,
    "OdometerIn" :"repair_order_serviced_vehicle_odometer_in",
    "OdometerOut": "repair_order_odometer_out" ,
    "ReferenceNumber": "repair_order_reference_number" ,
    "PrintedDate" : "repair_order_receipt_printed_datetime" ,
    "TaxExempt": "repair_order_snapshot_is_tax_exempt" ,
    "Notes": "repair_order_aggr_notes" ,
    "Observations": "repair_order_observation_text_area" ,
    "CreatedAsEstimate": "repair_order_created_as_estimate"  ,
    "MarginPct": "repair_order_snapshot_margin_pct" ,
    "HazWasteAmt": "repair_order_snapshot_haz_waste_amount",
    "LaborSale":"repair_order_snapshot_labor_sale_amount" ,
    "PartsSale":"repair_order_snapshot_parts_sale_amount" ,
    "ShopSuppliesAmt":"repair_order_snapshot_supply_from_shop_amount" ,
    "TaxAmtHazMat":"repair_order_snapshot_tax_haz_material_amount" ,
    "TaxAmtShopSupplies":"repair_order_snapshot_tax_supply_from_shop_amount" ,
    "TotalTaxAmt":"repair_order_snapshot_total_tax_amount" ,
    "BalanceDueAdjustment": "repair_order_snapshot_balance_due_adjusted" ,
    "DiscountAmt": "repair_order_snapshot_discounted_amount",
    "PartDiscountDescriptionId": "repair_order_snapshot_part_discounted_desc_id",
    "LaborRateDescriptionId":"repair_order_snapshot_labor_discounted_desc_id" ,
    "OrderTotal":"repair_order_snapshot_order_total_amount" ,
    "CalculateHazWasteCost":"repair_order_snapshot_calc_haz_waste_cost"  ,
    "CalculateShopSuppliesCost":"repair_order_snapshot_calc_shop_supply_cost" ,
    "EngineHoursIn":"repair_order_serviced_vehcle_engine_hours_in",
    "EngineHoursOut":"repair_order_serviced_vehcle_engine_hours_out" ,
    "LastChangeDate":"repair_order_last_updated_datetime",
    "AppointmentRequestUid":"repair_order_appointment_request_uid" })

In [3]:
df_repairorder_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97281 entries, 0 to 97280
Data columns (total 42 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   repair_order_id                                    97281 non-null  Int64  
 1   repair_order_phase_id                              97281 non-null  Int64  
 2   repair_order_cust_id                               95430 non-null  Int64  
 3   repair_order_vehicle_id                            93772 non-null  Int64  
 4   repair_order_serviced_vehicle_location             97278 non-null  string 
 5   repair_order_service_status                        97281 non-null  string 
 6   repair_order_scheduled_start_datetime              97147 non-null  string 
 7   repair_order_billed_hours                          97281 non-null  Float64
 8   repair_order_promise_datetime                      97170 non-null  string 
 9   repair

In [ ]:
db_engine_repairorder=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
# writing data (repair_order_final, the formatted verison) into the new tbl repairorders_new_01
with db_engine_repairorder.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_repairorder_final.to_sql('repairorders_new_01',conn, index=False,if_exists='append')

In [12]:
# dataframe(df)
json_file_path = '/Users/stephenwang/Documents/76prolubeplus.com/mitchell1_db_jsons/estimate_Mitchell1_20221222.json'
df_estimate=pd.read_json(json_file_path)
df_estimate=df_estimate.convert_dtypes()


In [13]:

df_estimate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95644 entries, 0 to 95643
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EstimateId            95644 non-null  Int64  
 1   RepairOrderId         95644 non-null  Int64  
 2   EstimatedLaborCost    95644 non-null  Float64
 3   EstimatedPartsCost    95644 non-null  Float64
 4   EstimatedHours        95483 non-null  Float64
 5   EstimatedTaxAmount    95644 non-null  Float64
 6   EstimatedHazMatCost   95644 non-null  Float64
 7   EstShopSuppliesCost   95644 non-null  Int64  
 8   EstimatedDiscount     95644 non-null  Float64
 9   timestamp             95644 non-null  string 
 10  EstimatedTireFeeCost  62070 non-null  Int64  
 11  RecordVersion         95644 non-null  Int64  
 12  LastChangeDate        95644 non-null  string 
dtypes: Float64(6), Int64(5), string(2)
memory usage: 10.5 MB


In [14]:
#df_estimate_cut.info()
df_estimate_cut=df_estimate.loc[:, ['EstimateId', 'RepairOrderId', 
       'EstimatedLaborCost','EstimatedPartsCost', 'EstimatedHours', 'EstimatedTaxAmount',
       'EstimatedHazMatCost', 'EstShopSuppliesCost', 'EstimatedDiscount',
       'LastChangeDate']]
df_estimate_final=df_estimate_cut.rename(
    columns={'EstimateId':'estimate_id', 
             'RepairOrderId':'estimate_repair_order_id', 
             'EstimatedLaborCost':'estimate_snapshot_labor_cost',
             'EstimatedPartsCost':'estimate_snapshot_parts_cost', 
             'EstimatedHours':'estimate_billed_hours', 'EstimatedTaxAmount':'estimate_snapshot_tax_amount',
             'EstimatedHazMatCost':'estimate_snapshot_haz_mat_amount',
             'EstShopSuppliesCost':'estimate_snapshot_shop_supply_cost', 
             'EstimatedDiscount':'estimate_snapshot_discount_amount',
             'LastChangeDate':'estimate_last_updated_date'
        
    }
)


In [16]:
df_estimate_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95644 entries, 0 to 95643
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   estimate_id                         95644 non-null  Int64  
 1   estimate_repair_order_id            95644 non-null  Int64  
 2   estimate_snapshot_labor_cost        95644 non-null  Float64
 3   estimate_snapshot_parts_cost        95644 non-null  Float64
 4   estimate_billed_hours               95483 non-null  Float64
 5   estimate_snapshot_tax_amount        95644 non-null  Float64
 6   estimate_snapshot_haz_mat_amount    95644 non-null  Float64
 7   estimate_snapshot_shop_supply_cost  95644 non-null  Int64  
 8   estimate_snapshot_discount_amount   95644 non-null  Float64
 9   estimate_last_updated_date          95644 non-null  string 
dtypes: Float64(6), Int64(3), string(1)
memory usage: 8.1 MB


In [17]:
db_engine_estimate=create_engine(f'mssql+pyodbc:///?odbc_connect={sql_alchemy_string}')

In [ ]:
# writing data (repair_order_final, the formatted verison) into the new tbl repairorders_new_01
with db_engine_estimate.connect() as conn:
    conn.execute('use prolube76DBSTG01;')
    df_estimate_final.to_sql('estimates_new_01',conn, index=False,if_exists='append')